In [68]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

stopWords = set(word.strip().lower() for word in open("stop_words_en.txt"))

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    
    for word in words:                
        key = word.lower()
        if key not in stopWords and key[0].isalpha():
            print "%s\t%s" % (key, word)

Overwriting mapper.py


In [69]:
%%writefile reducer.py

import sys

current_key = None
name_sum = 0
word_sum = 0

Overwriting reducer.py


In [70]:
%%writefile -a reducer.py

def isName(word):
    return len(word) > 1 and word[0].isupper() and word[1:].islower()
    
def isNameProportion():
    return (100.0*name_sum/word_sum >= 99.5)

for line in sys.stdin:
    try:
        key, word = line.strip().split('\t', 1)        
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            if isNameProportion():
                print "%d\t%s" % (name_sum, current_key)
        word_sum = 0
        name_sum = 0
        current_key = key    
    word_sum += 1
    if isName(word):
        name_sum += 1      

if current_key:
    if isNameProportion():
        print "%d\t%s" % (name_sum, current_key)

Appending to reducer.py


In [71]:
%%writefile mapper1.py

import sys

for line in sys.stdin:
    print line

Overwriting mapper1.py


In [72]:
%%writefile reducer1.py

import sys

index = 0
for line in sys.stdin:
    try:
        count, word = line.strip().split('\t', 1)        
    except ValueError as e:
        continue
    index += 1
    if index == 5:
        print "%s\t%s"%(word, count)       

Overwriting reducer1.py


In [61]:
%%bash

OUT_DIR=namecount_result
RES_DIR=namecount_sorted

NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null
hdfs dfs -rm -r -skipTrash ${RES_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Streaming name count" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null   

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Streaming name count sort" \
    -D mapreduce.job.reduces=1 \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options=-nr \
    -files mapper1.py,reducer1.py \
    -mapper "python mapper1.py" \
    -reducer "python reducer1.py" \
    -input namecount_result \
    -output ${RES_DIR} > /dev/null

hdfs dfs -cat ${RES_DIR}/part-00000 | head

john	5762


17/11/30 19:41:03 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/30 19:41:04 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/30 19:41:10 INFO mapred.FileInputFormat: Total input files to process : 1
17/11/30 19:41:11 INFO mapreduce.JobSubmitter: number of splits:2
17/11/30 19:41:12 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1512056254976_0021
17/11/30 19:41:13 INFO impl.YarnClientImpl: Submitted application application_1512056254976_0021
17/11/30 19:41:13 INFO mapreduce.Job: The url to track the job: http://957a16de490c:8088/proxy/application_1512056254976_0021/
17/11/30 19:41:13 INFO mapreduce.Job: Running job: job_1512056254976_0021
17/11/30 19:41:33 INFO mapreduce.Job: Job job_1512056254976_0021 running in uber mode : false
17/11/30 19:41:33 INFO mapreduce.Job:  map 0% reduce 0%
17/11/30 19:42:01 INFO mapreduce.Job:  map 19% reduce 0%
17/11/30 19:42:08 INFO mapreduce.Job:  map 31% reduce 0%
17/11/30 19:42:14 INFO 